In [213]:
%load_ext autoreload
%autoreload 2

import numpy as np
from account import Binance
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
import cvxpy as cp
from utils.logging import get_logger
from utils.data_helper import *
from utils.db import *
from strategy_v3.Strategy import ExchangeArbitrageStrategy

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.options.display.float_format = "{:,.4f}".format
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Expermental Arbitrage strategy in Binance

1. Get bid/ask price for all active currency pairs in Binance and presents in a matrix $Q$

2. Transform the price to negative log price (Given converison from currency A->B->C = log(p1*p2) ~= log(p1) + log(p2))

3. Solve the optimization follow classic Traveling Salesmen Problem (TSP), but removing the constraints that all nodes needs to be visited once

- We want to find a closed loop where the sum of path values are negative

- Input $X$ is the nxn binary matrix (n is number of assets), 1 represents trade from currency x -> y

- Minimize $X$ dot $Q$

Reference: https://nbviewer.org/github/rcroessmann/sharing_public/blob/master/arbitrage_identification.ipynb


In [214]:
strategy = ExchangeArbitrageStrategy(zero_fees=True, trades_num=3)
strategy.set_strategy_id("qa")
strategy.load_data()
strategy.optimize()
df_trades = strategy.df_trades
df_trades

2025-01-17 03:16:48,485 - 506546 - INFO - {'USDT': 50, 'ETH': 0.01539456264, 'BTC': 0.00052979531}


                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Jan 17 03:16:48 AM: Your problem has 163216 variables, 5 constraints, and 0 parameters.
(CVXPY) Jan 17 03:16:48 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 17 03:16:48 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 17 03:16:48 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 17 03:16:48 AM: Compiling problem (target solver=SCIPY).
(CVXPY) Jan 17 03:16:48 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffi

2025-01-17 03:16:55,096 - qa - INFO - CVXPY - Status: optimal
2025-01-17 03:16:55,096 - qa - INFO - CVSPY - Optimal value: -0.0004248364320833531
2025-01-17 03:16:55,098 - qa - INFO - Total PNL: -0.3572%
2025-01-17 03:16:55,218 - qa - INFO - 
+---------+--------------+------------+---------+
|   group |   gross_pnl% |   net_pnl% |   count |
|---------+--------------+------------+---------|
|       1 |    0.0424927 |  -0.357152 |       3 |
+---------+--------------+------------+---------+
2025-01-17 03:16:55,218 - qa - INFO - Net pnl is too small, end here.


,from_asset,to_asset,mkt_price,fee,group,order,mkt_price_w_fee,symbol,status,baseAsset,quoteAsset,stepSize,tickSize,qty_decimal,price_decimal,bidPrice,bidQty,askPrice,askQty,makerCommission,takerCommission,side,count,price_time,zero_fees
0,ETH,TRX,"13,906.2717",0.0010,1,1,"13,892.3655",TRXETH,TRADING,TRX,ETH,1.0000,0.0000,0,8,0.0001,"1,011.0000",0.0001,637.0000,0.0010,0.0010,BUY,1,2025-01-17 03:16:48.568376+08:00,True
1,TRX,TRY,8.4910,0.0015,1,2,8.4783,TRXTRY,TRADING,TRX,TRY,1.0000,0.0010,0,3,8.4910,"1,236.0000",8.4920,"45,992.0000",0.0010,0.0015,SELL,1,2025-01-17 03:16:48.568376+08:00,True
2,TRY,ETH,0.0000,0.0015,1,3,0.0000,ETHTRY,TRADING,ETH,TRY,0.0001,1.0000,4,0,"118,012.0000",0.1586,"118,028.0000",0.1586,0.0010,0.0015,BUY,1,2025-01-17 03:16:48.568376+08:00,True


# Summary of strategy

In [217]:
db = duck("binance_arb")
db.query('''
    select
        price_time
        , 100 * (product(mkt_price_w_fee) - 1) as pnl_net
        , 100 * (product(mkt_price) - 1) as pnl_gross
        , pnl_gross - pnl_net as fees
        , count(1) as num_trades
        , string_agg(from_asset, ', ') as ccy
        , bool_or(from_asset in ('USDT', 'ETH', 'BTC')) as tradable
    from trades
    group by price_time       
    order by price_time
''')

,price_time,pnl_net,pnl_gross,fees,num_trades,ccy,tradable
0,2025-01-15 00:37:06.000888,0.0419,0.2372,0.1954,3,"PLN, USDT, USDC",True
1,2025-01-15 07:57:04.912990,0.2943,0.4902,0.1959,3,"PLN, USDT, USDC",True
2,2025-01-15 21:30:05.011390,0.1923,0.4935,0.3012,5,"ARS, BTC, EUR, EURI, USDT",True
3,2025-01-15 21:33:06.989914,-0.0000,-0.0000,0.0000,3,"FDUSD, USDC, USDT",True
4,2025-01-15 22:56:05.158265,0.0589,0.3597,0.3008,3,"EGLD, USDT, RON",True
5,2025-01-15 23:01:05.812660,0.0035,0.3041,0.3006,3,"EGLD, USDT, RON",True
6,2025-01-16 01:09:04.844629,-0.0000,-0.0000,0.0000,3,"FDUSD, USDC, USDT",True
7,2025-01-16 02:25:04.941040,0.0273,0.3280,0.3007,3,"TRX, WIN, USDT",True
8,2025-01-16 03:35:05.034956,0.0001,0.3007,0.3006,3,"TRX, WIN, USDT",True
9,2025-01-16 04:18:04.071861,-0.0000,-0.0000,0.0000,3,"FDUSD, USDC, USDT",True


# Trade the arbitrage pair(s)
- Trade the pairs with highest pnl
- The arbitrage loop starts with one existing currency in current portfolios

- commission are included in quote quantity?

In [92]:
trade_currency = {
    'USDT': 50,
    'ETH':  0.01539456264,
    'BTC': 0.00052979531,
}

In [119]:
client = Binance().client
balance = client.get_account()
balance = pd.DataFrame(balance['balances'])
balance['free'] = balance['free'].astype(float)
balance['locked'] = balance['locked'].astype(float)
balance = balance[balance['free'] > 0]
balance

,asset,free,locked
0,BTC,0.0701,0.0000
2,ETH,1.8632,0.0000
4,BNB,0.0000,0.0000
11,USDT,"9,758.6666","2,416.7007"
22,TRX,0.0000,0.0000
61,XRP,2.6387,0.0000
86,ADA,0.0734,0.0000
88,XLM,99.9000,0.0000
134,IOTX,8.1410,0.0000
143,DENT,0.7540,0.0000
